In [1]:
import os
import tensorflow as tf

from utils import (
    DataLoader,
    PerformanceMetrics,
    get_subject_ids,
    perform_loocv,
    reset_weights
)
from models import NDDNet

# ... Hide tensorflow debug messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# ... Disable GPU
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"


2022-10-23 04:15:41.705240: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Config

In [2]:
DATA_DIR = "../data/gaitndd/"

FS = 300
N_EPOCHS = 300
LEARNING_RATE = 3e-4
GRF_SEGMENT_LEN = 120
VAL_PERCENTAGE = 0.3
MODE = "combined"

# Utils

In [ ]:
all_subject_ids = get_subject_ids(DATA_DIR, "ndd")
dataloader = DataLoader(DATA_DIR, all_subject_ids)
metrics = PerformanceMetrics()


# Model

In [4]:
network = NDDNet(
    n_classes=2,
    n_conv_blocks=3,
    n_mlp_layers=0,
    kernel_size=3,
    conv_channel_width=16,
    mlp_channel_width=32,
    mode="combined"
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)


# ALS vs HC

In [6]:
DISEASE = "als"

metrics = PerformanceMetrics()
subject_ids = get_subject_ids(DATA_DIR, DISEASE)

for test_subject in ["als7"]:
    print(f"Training on {test_subject} ... ")
    y_true, y_pred, model = perform_loocv(
        test_subject=test_subject,
        val_percentage=VAL_PERCENTAGE,
        subject_ids=subject_ids,
        dataloader=dataloader,
        network=network,
        loss=loss,
        optimizer=optimizer,
        n_epochs=N_EPOCHS
    )

    metrics.add_entry(test_subject, y_true, y_pred)
    # model.save_weights(f"../weights/{DISEASE}/{test_subject}", save_format="h5")

    break

metrics.print_metrics()



Training on als7 ... 


                   Performace Metrics                    
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Subject ID ┃ Precision ┃ Recall ┃ F1-Score ┃ Accuracy ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ als7       │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ ALL        │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
└────────────┴───────────┴────────┴──────────┴──────────┘

# HD vs HC

In [5]:
DISEASE = "hunt"

metrics = PerformanceMetrics()
subject_ids = get_subject_ids(DATA_DIR, DISEASE)

for test_subject in ["control11", "control14", "control9"]:
    print(f"Training on {test_subject} ... ")
    y_true, y_pred, model = perform_loocv(
        test_subject=test_subject,
        val_percentage=VAL_PERCENTAGE,
        subject_ids=subject_ids,
        dataloader=dataloader,
        network=network,
        loss=loss,
        optimizer=optimizer,
        n_epochs=N_EPOCHS
    )

    metrics.add_entry(test_subject, y_true, y_pred)
    model.save_weights(f"../weights/{DISEASE}/{test_subject}", save_format="h5")

    # break

metrics.print_metrics()



Training on control11 ... 


2022-10-23 04:16:59.867939: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-23 04:16:59.867967: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Andromeda
2022-10-23 04:16:59.867973: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Andromeda
2022-10-23 04:16:59.868093: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 520.56.6
2022-10-23 04:16:59.868107: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.56.6
2022-10-23 04:16:59.868110: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 520.56.6
2022-10-23 04:16:59.868355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions i

Training on control14 ... 
Training on control9 ... 


                   Performace Metrics                    
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Subject ID ┃ Precision ┃ Recall ┃ F1-Score ┃ Accuracy ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ control11  │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ control14  │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ control9   │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ ALL        │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
└────────────┴───────────┴────────┴──────────┴──────────┘

# PD vs HC

In [9]:
DISEASE = "park"

metrics = PerformanceMetrics()
subject_ids = get_subject_ids(DATA_DIR, DISEASE)

for test_subject in ["park7"]:
    print(f"Training on {test_subject} ... ")
    y_true, y_pred, model = perform_loocv(
        test_subject=test_subject,
        val_percentage=VAL_PERCENTAGE,
        subject_ids=subject_ids,
        dataloader=dataloader,
        network=network,
        loss=loss,
        optimizer=optimizer,
        n_epochs=N_EPOCHS
    )

    metrics.add_entry(test_subject, y_true, y_pred)
    model.save_weights(f"../weights/{DISEASE}/{test_subject}", save_format="h5")

    # break

metrics.print_metrics()



Training on park7 ... 


                   Performace Metrics                    
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Subject ID ┃ Precision ┃ Recall ┃ F1-Score ┃ Accuracy ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ park7      │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ ALL        │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
└────────────┴───────────┴────────┴──────────┴──────────┘

# NDD vs HC

In [12]:
DISEASE = "ndd"

metrics = PerformanceMetrics()
subject_ids = get_subject_ids(DATA_DIR, DISEASE)

for test_subject in ["park3"]:
    print(f"Training on {test_subject} ... ")
    y_true, y_pred, model = perform_loocv(
        test_subject=test_subject,
        val_percentage=VAL_PERCENTAGE,
        subject_ids=subject_ids,
        dataloader=dataloader,
        network=network,
        loss=loss,
        optimizer=optimizer,
        n_epochs=N_EPOCHS
    )

    metrics.add_entry(test_subject, y_true, y_pred)
    model.save_weights(f"../weights/{DISEASE}/{test_subject}", save_format="h5")

    # break

metrics.print_metrics()



Training on park3 ... 


                   Performace Metrics                    
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Subject ID ┃ Precision ┃ Recall ┃ F1-Score ┃ Accuracy ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ park3      │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
│ ALL        │ 1.000     │ 1.000  │ 1.000    │ 1.000    │
└────────────┴───────────┴────────┴──────────┴──────────┘